In [1]:
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
import sys

In [2]:
src_path = '/Users/kirillsafronov/Documents/Python/Lab/DNN'
if src_path not in sys.path:
    sys.path.append(src_path)

In [3]:
from src.utils import set_det_pos, get_detector_imgs
from src.DNN import Fourier_DNN, Trainer, new_Fourier_DNN

In [4]:
# Загрузка датасета
BATCH_SIZE = 32
IMG_SIZE = 28 * 3
N_pixels = 200
PADDING = (N_pixels - IMG_SIZE) // 2

transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((IMG_SIZE,IMG_SIZE))])
train_dataset = torchvision.datasets.MNIST("./data", train=True, transform=transform, download=True)
val_dataset = torchvision.datasets.MNIST("./data", train=False, transform=transform, download=True)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [6]:
pixel_size = 10e-6 #m
wl = 532e-9 #m 
distance_between_layers = 5e-3

In [7]:
det_size = 20
det_pad = (N_pixels - 7*det_size)//2
detector_pos = set_det_pos(det_size, det_pad, det_pad)

In [8]:
labels_image_tensors = get_detector_imgs(detector_pos, N_pixels).to(device)

In [9]:
def custom_loss(imgs, labels):
    full_int = imgs.sum(dim=(1,2))
    loss = 1 - (imgs*labels_image_tensors[labels]).sum(dim=(1,2))/full_int
    return loss.mean()

In [10]:
f = 100e-3
model = new_Fourier_DNN(5, wl, N_pixels, pixel_size, distance_between_layers, f).to(device)
#criterion = torch.nn.MSELoss(reduction='sum').to(device)
#criterion = torch.nn.CrossEntropyLoss().to(device)
criterion = custom_loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [11]:
trainer = Trainer(model, detector_pos, PADDING, device) 

In [12]:
res = trainer.train(criterion, optimizer, train_dataloader, val_dataloader, epochs = 2)

100%|██████████| 313/313 [01:34<00:00,  3.32it/s]

Epoch=1 train loss=0.4935, test loss=0.4792
train acc=0.8167, test acc=0.8339
-----------------------
